In [1]:
import os
import time
import threading

#### Current Thread

In [5]:
print('Program enter')
print(f'Process id: {os.getpid()}')
print(f'Current Thread name: {threading.current_thread().name}')
print('Sleeping...')
time.sleep(5) # Current thread will go to sleep state
threading.current_thread().name = 'MyThread'
print(f'Current Thread name: {threading.current_thread().name}')
print('Program exit')

Program enter
Process id: 856
Current Thread name: MainThread
Sleeping...
Current Thread name: MyThread
Program exit


#### Create a new Thread
- Instantiate the Thread class and set the target function to be executed 
- Write a sub-class of Thread and instantiate the sub-class 

In [2]:
def run():
    for i in range(1,6):
        print(f'{i} --> Thread name: {threading.current_thread().name}, Process id: {os.getpid()}')
        time.sleep(1)

In [18]:
print(f'Main thread enter, Process id: {os.getpid()}')

t1 = threading.Thread(target=run, name='Thread-01')
t1.start()

print(f't1 is alive: {t1.is_alive()}')

print('Main is waiting')
t1.join() # current thread (main) goes to wait state, gets notified when t1 is dead
print(f't1 is alive: {t1.is_alive()}')
print('Main is no more waiting')

print('Main thread exit')

Main thread enter, Process id: 967
1 --> Thread name: Thread-01, Process id: 967
t1 is alive: True
Main is waiting
2 --> Thread name: Thread-01, Process id: 967
3 --> Thread name: Thread-01, Process id: 967
4 --> Thread name: Thread-01, Process id: 967
5 --> Thread name: Thread-01, Process id: 967
t1 is alive: False
Main is no more waiting
Main thread exit


In [23]:
print(f'Main thread enter, Process id: {os.getpid()}')

t1 = threading.Thread(target=run, name='Thread-01')
t2 = threading.Thread(target=run, name='Thread-02')

t1.start()
t2.start()

t1.join()
t2.join()

print('Main thread exit')

Main thread enter, Process id: 967
1 --> Thread name: Thread-01, Process id: 967
1 --> Thread name: Thread-02, Process id: 967
2 --> Thread name: Thread-01, Process id: 967
2 --> Thread name: Thread-02, Process id: 967
3 --> Thread name: Thread-01, Process id: 967
3 --> Thread name: Thread-02, Process id: 967
4 --> Thread name: Thread-01, Process id: 967
4 --> Thread name: Thread-02, Process id: 967
5 --> Thread name: Thread-01, Process id: 967
5 --> Thread name: Thread-02, Process id: 967
Main thread exit


#### Create a Thread whose target function accepts numbers, adds them and display the sum along with the Thread name

In [27]:
def add(*args):
    print(f'Thread name: {threading.current_thread().name}, Sum: {sum(args)}')
    
t1 = threading.Thread(target=add, name='First', args=(1,2,3,4,5))
t2 = threading.Thread(target=add, name='Second', args=(11,22,33))

t1.start()
t2.start()

Thread name: First, Sum: 15
Thread name: Second, Sum: 66


#### Create Thread by Inheritance
- Write a sub-class of Thread
- Target method by default is instance method 'run', we need to override it
- In case the method is not named as run, then the target is set explicitly through the run attribute (t.run = method)
- Arguments are passed through args attribute (t.args) and  accessed through 'self' (self.args)

In [5]:
class MyThread(threading.Thread):
    def run(self):
        for i in range(1,6):
            print(f'{i} --> Thread name: {threading.current_thread().name}, Process id: {os.getpid()}')
            time.sleep(0.5)

#------------------------------------

t1 = MyThread()
t1.name = 'Thread-01'
t1.start()

1 --> Thread name: Thread-01, Process id: 1090
2 --> Thread name: Thread-01, Process id: 1090
3 --> Thread name: Thread-01, Process id: 1090
4 --> Thread name: Thread-01, Process id: 1090
5 --> Thread name: Thread-01, Process id: 1090


In [7]:
class MyThread(threading.Thread):
    def do_something(self):
        for i in range(1,6):
            print(f'{i} --> Thread name: {threading.current_thread().name}, Process id: {os.getpid()}')
            time.sleep(0.5)

#------------------------------------

t1 = MyThread()
t1.name = 'Thread-01'
t1.run = t1.do_something # set the target
t1.start()

1 --> Thread name: Thread-01, Process id: 1090
2 --> Thread name: Thread-01, Process id: 1090
3 --> Thread name: Thread-01, Process id: 1090
4 --> Thread name: Thread-01, Process id: 1090
5 --> Thread name: Thread-01, Process id: 1090


**Write a sub-class of Thread named as AddThread**
- The target method would sum the arguments and display the sum along with the thread name

In [2]:
class AddThread(threading.Thread):
    def run(self):
        print(f'Thread name: {threading.current_thread().name}, Sum: {sum(self.args)}')

#--------------------------------------------

t1 = AddThread()
t1.name = 'First'
t1.args = (1,2,3,4,5)

t2 = AddThread()
t2.name = 'Second'
t2.args = (10,20,30)

t1.start()
t2.start()

Thread name: First, Sum: 15
Thread name: Second, Sum: 60


#### Thread assignment
1. It will have an instance field named 'data', initialized to 0
2. It will have an instance method named 'do_something'
3. The method will increment data by 1 and display the name of the current thread and value of data
4. This method ('do_something') will be target for a Thread
5. Terms to be remembered: Race condition, Thread synchronization, Mutex, Semaphore, Atomic operation, Critical section 

**Note: We need not use inheritance approach**

In [40]:
class Resource1:
    def __init__(self):
        self.data = 0
    
    def do_something(self):
        self.data += 1
        for i in range(1000000):
            pass
        print(f'{threading.current_thread().name} - {self.data}')

**Thread safe code**

In [41]:
class Resource2:
    def __init__(self):
        self.data = 0
        self.lock = threading.Lock()
    
    def do_something(self):
        with self.lock:
            self.data += 1
            for i in range(1000000):
                pass
            print(f'{threading.current_thread().name} --> {self.data}')
        
        
        '''
        self.lock.acquire()
        self.data += 1
        for i in range(1000000):
            pass
        print(f'{threading.current_thread().name} --> {self.data}')
        self.lock.release()
        '''

In [43]:
#r = Resource1() # Not Thread safe
r = Resource2() # Thread safe


t1 = threading.Thread(target = r.do_something, name='First')
t2 = threading.Thread(target = r.do_something, name='Second')
t3 = threading.Thread(target = r.do_something, name='Third')

t1.start()
t2.start()
t3.start()

First --> 1
Second --> 2
Third --> 3


In [45]:
#r = Resource1()
r = Resource2()

for i in range(1,11):
    t = threading.Thread(target = r.do_something, name=f'Thread-{i}')
    t.start()

Thread-1 --> 1
Thread-2 --> 2
Thread-3 --> 3
Thread-4 --> 4
Thread-5 --> 5
Thread-6 --> 6
Thread-7 --> 7
Thread-8 --> 8
Thread-9 --> 9
Thread-10 --> 10
